In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import gensim.downloader as api
from tensorflow.keras.layers import Bidirectional

In [4]:
# Load dataset
file_path = "/Users/celinewu/Documents/GitHub/2024-25c-fai2-adsai-group-group16/Task_4/ver_2_FINAL_DATASET.xlsx"
df = pd.read_excel(file_path)

In [5]:
# Extract text and labels
sentences = df["Sentence"].astype(str).tolist()
labels = df["main_category"].astype(str).tolist()

# Extract Sentiment Scores
sentiment_scores = df["Sentiment_Score"].values.reshape(-1, 1)

# Extract TF-IDF Feature as an Additional Feature
tfidf_features = np.array([np.fromstring(vec.strip("[]"), sep=' ') for vec in df["TF_IDF"]])

# Normalize Features
scaler = MinMaxScaler()
sentiment_scores = scaler.fit_transform(sentiment_scores)
tfidf_features = scaler.fit_transform(tfidf_features)

/var/folders/kz/slhyr0gd4qj4nrryyswvqv4c0000gn/T/ipykernel_86244/2282603795.py:9: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  tfidf_features = np.array([np.fromstring(vec.strip("[]"), sep=' ') for vec in df["TF_IDF"]])


In [6]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(sentences)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")


In [7]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)


In [8]:
# Split dataset
X_train, X_test, y_train, y_test, X_train_sent, X_test_sent, X_train_tfidf, X_test_tfidf = train_test_split(
    padded_sequences, encoded_labels, sentiment_scores, tfidf_features,
    test_size=0.2, random_state=42, stratify=encoded_labels
)

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


In [9]:
# Load GloVe embeddings
print("Loading GloVe embeddings...")
glove_model = api.load("glove-wiki-gigaword-100")

# Create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

# Define text input and embedding layer
text_input = Input(shape=(max_length,), name="text_input")
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                      weights=[embedding_matrix], input_length=max_length, 
                      trainable=True)(text_input)



Loading GloVe embeddings...


/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-03-04 16:34:10.275020: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-03-04 16:34:10.275055: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-04 16:34:10.275059: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-04 16:34:10.275073: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-04 16:34:10.275085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefin

In [10]:
# Define Biderectional GRU layers
rnn_layer = Bidirectional(GRU(128, return_sequences=True, recurrent_dropout=0.2))(embedding)
rnn_layer = Bidirectional(GRU(64, recurrent_dropout=0.2))(rnn_layer)
dense_text = Dense(64, activation='relu')(rnn_layer)

# Define Sentiment Score input
sentiment_input = Input(shape=(1,), name="sentiment_input")
sentiment_dense = Dense(8, activation='relu')(sentiment_input)

# Define TF-IDF input
tfidf_input = Input(shape=(tfidf_features.shape[1],), name="tfidf_input")
tfidf_dense = Dense(32, activation='relu')(tfidf_input)


In [11]:
# Merge inputs
merged = Concatenate()([dense_text, sentiment_dense, tfidf_dense])
output = Dense(num_classes, activation='softmax')(merged)


In [12]:
# Define final model
model = Model(inputs=[text_input, sentiment_input, tfidf_input], outputs=output)
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(encoded_labels), y=encoded_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


In [13]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("sentiment_tfidf_it4.keras", monitor='val_accuracy', save_best_only=True)

# Train the model
print("Training model...")
history = model.fit(
    [X_train, X_train_sent, X_train_tfidf], y_train,
    epochs=30, batch_size=64, validation_data=([X_test, X_test_sent, X_test_tfidf], y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping, model_checkpoint]
)

Training model...
Epoch 1/30


2025-03-04 16:34:12.217255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


70/70 ━━━━━━━━━━━━━━━━━━━━ 381s 5s/step - accuracy: 0.1282 - loss: 0.5663 - val_accuracy: 0.1362 - val_loss: 0.4479
Epoch 2/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 372s 5s/step - accuracy: 0.1432 - loss: 0.4180 - val_accuracy: 0.1461 - val_loss: 0.4384
Epoch 3/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 371s 5s/step - accuracy: 0.1397 - loss: 0.4144 - val_accuracy: 0.1487 - val_loss: 0.4363
Epoch 4/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 371s 5s/step - accuracy: 0.1573 - loss: 0.4125 - val_accuracy: 0.1586 - val_loss: 0.4327
Epoch 5/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 376s 5s/step - accuracy: 0.1755 - loss: 0.4112 - val_accuracy: 0.1694 - val_loss: 0.4317
Epoch 6/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 376s 5s/step - accuracy: 0.1958 - loss: 0.4089 - val_accuracy: 0.1801 - val_loss: 0.4298
Epoch 7/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 372s 5s/step - accuracy: 0.2105 - loss: 0.4064 - val_accuracy: 0.1927 - val_loss: 0.4290
Epoch 8/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 373s 5s/step - accuracy: 0.2181 - loss: 0.4076 - val_accuracy: 0.2276 - val_loss: 0.

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate([X_test, X_test_sent, X_test_tfidf], y_test)
print(f"Test Accuracy: {test_acc:.4f}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 273ms/step - accuracy: 0.3570 - loss: 0.3774
Test Accuracy: 0.3351


In [15]:
# Save tokenizer
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [16]:
# Generate F1-score report
from sklearn.metrics import classification_report

# Get predictions
y_pred = model.predict([X_test, X_test_sent, X_test_tfidf])

# Convert predictions from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))


35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 279ms/step

Classification Report:
              precision    recall  f1-score   support

       anger       0.25      0.38      0.30       159
     disgust       0.36      0.42      0.39       159
        fear       0.45      0.40      0.43       159
   happiness       0.52      0.47      0.49       160
     neutral       0.23      0.16      0.19       160
     sadness       0.30      0.19      0.23       159
    surprise       0.26      0.33      0.29       160

    accuracy                           0.34      1116
   macro avg       0.34      0.34      0.33      1116
weighted avg       0.34      0.34      0.33      1116



In [17]:
# Generate F1-score report
from sklearn.metrics import classification_report

# Get predictions
y_pred = model.predict([X_test, X_test_sent, X_test_tfidf])

# Convert predictions from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))


35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 269ms/step

Classification Report:
              precision    recall  f1-score   support

       anger       0.25      0.38      0.30       159
     disgust       0.36      0.42      0.39       159
        fear       0.45      0.40      0.43       159
   happiness       0.52      0.47      0.49       160
     neutral       0.23      0.16      0.19       160
     sadness       0.30      0.19      0.23       159
    surprise       0.26      0.33      0.29       160

    accuracy                           0.34      1116
   macro avg       0.34      0.34      0.33      1116
weighted avg       0.34      0.34      0.33      1116

